In [25]:
import os
import librosa
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import distutils as _distutils

In [26]:
model=tf.keras.models.load_model('Trained_model.h5')

In [27]:
classes=["blues","classical","country","disco","hiphop","jazz","metal","pop","reggae","rock"]

# Single Audio Processing

In [51]:
from tensorflow.image import resize

def load_and_preprocess(file_path,target_shape=[210,210]):
    data=[]
    audio_data,sample_rate= librosa.load(file_path,sr=None)
    chunk_duration=4
    overlap_duration=2

    chunk_sample=chunk_duration * sample_rate
    overlap_sample=overlap_duration * sample_rate

    num_chunks = int(np.ceil((len(audio_data) - chunk_sample) / (chunk_sample - overlap_sample))) + 1

    # Iterate over chunks
    for i in range(num_chunks):
                        start = i * (chunk_sample - overlap_sample)
                        end = start + chunk_sample
                        chunk = audio_data[start:end]

                        # Check if the chunk is valid (not empty)
                        if len(chunk) < chunk_sample:
                            continue

                        # Compute the Mel spectrogram
                        melspectrogram = librosa.feature.melspectrogram(y=chunk, sr=sample_rate)

                        # Resizing the Mel spectrogram
                        melspectrogram = resize(np.expand_dims(melspectrogram, axis=-1), target_shape)

                        # Append to data
                        data.append(melspectrogram)
    return np.array(data)

In [75]:
file_path= "./Music_Genre_Dataset/genres_original/pop/pop.00001.wav"
X_Test=load_and_preprocess(file_path)

In [76]:
y,sr=librosa.load(file_path,sr=44100)

In [77]:
X_Test

array([[[[2.53108826e+02],
         [4.25557739e+02],
         [5.84998657e+02],
         ...,
         [1.15674881e+02],
         [3.00238220e+02],
         [5.20246521e+02]],

        [[6.90735107e+02],
         [1.39163757e+03],
         [1.56292004e+03],
         ...,
         [9.49063049e+02],
         [1.45719629e+03],
         [1.21455066e+03]],

        [[1.29370923e+03],
         [2.70640259e+03],
         [2.88536108e+03],
         ...,
         [2.14642798e+03],
         [3.06383594e+03],
         [2.15859619e+03]],

        ...,

        [[7.87030995e-01],
         [8.09080839e-01],
         [5.91239989e-01],
         ...,
         [2.78168797e+00],
         [3.39298058e+00],
         [3.02073574e+00]],

        [[6.70391679e-01],
         [7.27317095e-01],
         [5.34541368e-01],
         ...,
         [2.61847734e+00],
         [2.98425364e+00],
         [2.52592039e+00]],

        [[5.80274105e-01],
         [6.62616313e-01],
         [4.90041494e-01],
         ...,
 

In [78]:
X_Test.shape

(14, 210, 210, 1)

In [79]:
def model_pred(X_Test):
    y_p=model.predict(X_Test)
    pred_cat= np.argmax(y_p,axis=1)
    unique_elements,counts=np.unique(pred_cat,return_counts=True)
    max_count= np.max(counts)
    max_elements= unique_elements[counts==max_count]
    return max_elements[0]

In [80]:

c_index=model_pred(X_Test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step


In [81]:
print("Model Prediction ::  MUSIC GENRE ------------->  ",classes[c_index])

Model Prediction ::  MUSIC GENRE ------------->   pop
